In [1]:
spark.version

'3.1.3'

In [2]:
#Importing Required Libraries 
import os
import shutil
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
pd.set_option("max_colwidth", 100)

from IPython.display import clear_output
clear_output(wait = False)

from google.cloud import storage

In [3]:
#Enabling EagerEval 
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

**->Discard irrelevant tweets (you can expect a significant number of tweets to be eliminated)**


**->Complete thorough EDA to identify which variables you can use to profile the Twitterers**

**----->A lot of variables are poorly populated and will have to be discarded**

In [4]:
# Specifiying the GCS Path 
path = 'gs://msca-bdp-tweets/final_project/'

In [5]:
#Reading the JSON File containing twitter data
tw_df = spark.read.json(path)

22/12/04 01:55:56 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
22/12/04 02:01:19 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
tw_df.limit(5)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_copyright,withheld_in_countries
null,Wed May 25 03:44:...,"[5, 140]","{[], null, [], [{...",null,"{[5, 201], {[], n...",0,false,low,null,1529307512608067584,1529307512608067584,AOC,1529305324951330816,1529305324951330816,138203134,138203134,false,en,null,null,0,null,null,null,null,null,0,0,,null,null,"<a href=""https://...",@AOC Quit using a...,1653450298822,true,@AOC Quit using a...,"{false, Fri Feb 0...",null,null
null,Wed May 25 03:44:...,null,"{[], null, [], [{...",null,null,0,false,low,null,1529307512490446849,1529307512490446849,null,null,null,null,null,false,en,null,false,0,null,null,null,null,null,0,0,,Deseret | https,null,"<a href=""https://...",How this Latter-d...,1653450298794,false,How this Latter-d...,"{false, Tue Mar 2...",null,null
null,Wed May 25 03:44:...,null,"{[{[0, 20], Texas...",null,null,0,false,low,null,1529307512620871681,1529307512620871681,null,null,null,null,null,false,en,null,false,0,null,null,null,null,null,0,0,,null,null,"<a href=""https://...",#TexasSchoolShoot...,1653450298825,false,#TexasSchoolShoot...,"{false, Tue Jul 1...",null,null
null,Wed May 25 03:44:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529307512775876608,1529307512775876608,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,MichaelKenKen,"{null, Wed May 25...","<a href=""http://t...",RT @MichaelKenKen...,1653450298862,false,We saw sitting po...,"{false, Sat Jul 2...",null,null
null,Wed May 25 03:44:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529307513232957440,1529307513232957440,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,CardinalBCupich,"{null, Wed May 25...","<a href=""http://t...",RT @CardinalBCupi...,1653450298971,false,"Today, a gunman w...","{false, Tue Feb 2...",null,null


In [7]:
print((tw_df.count(), len(tw_df.columns)))

22/12/04 02:04:04 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670090078315_0009_01_000018 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-s7mb.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 02:04:04.403]Container killed on request. Exit code is 143
[2022-12-04 02:04:04.404]Container exited with a non-zero exit code 143. 
[2022-12-04 02:04:04.419]Killed by external signal
.
22/12/04 02:04:04 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 18 for reason Container from a bad node: container_1670090078315_0009_01_000018 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-s7mb.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 02:04:04.403]Container killed on request. Exit code is 143
[2022-12-04 02:04:04.404]Container exited with a non-zero exit code 143. 
[2022-12-04 02:04:04.419]Killed by external signal
.
22/12/04 02:

(99992797, 40)


In [8]:
tw_df.columns

['coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'quoted_text',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_from',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'tweet_text',
 'user',
 'withheld_copyright',
 'withheld_in_countries']

In [9]:
tw_df.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

In [10]:
#Filtering useful and meaningful data base on language(English) and follower count(>10)
tw_df = tw_df.filter('lang="en"')\
.filter('user.followers_count>10')

In [11]:
tw_df.count()

22/12/04 02:11:34 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 29 for reason Container marked as failed: container_1670090078315_0009_01_000030 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wgps.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/12/04 02:11:34 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 26 for reason Container marked as failed: container_1670090078315_0009_01_000027 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wgps.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/12/04 02:11:34 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 29 on hub-msca-bdp-dphub-students-suyashlakhani-sw-wgps.c.msca-bdp-students.internal: Container marked as failed: container_1670090078315_0009_01_000030 on 

91182863

---> We can see that a lot of values have been discarded based on the initial filters. 

In [ ]:
#Checking the count of null values in all the columns
df_nullcount = tw_df.select([F.count(F.when(tw_df[c].isNull(), c)).alias(c) for c in tw_df.columns]).toPandas()
df_nullcount

22/12/04 02:25:21 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670090078315_0009_01_000033 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wwx8.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 02:25:21.168]Container killed on request. Exit code is 143
[2022-12-04 02:25:21.168]Container exited with a non-zero exit code 143. 
[2022-12-04 02:25:21.169]Killed by external signal
.
22/12/04 02:25:21 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670090078315_0009_01_000043 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wwx8.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 02:25:21.168]Container killed on request. Exit code is 143
[2022-12-04 02:25:21.169]Container exited with a non-zero exit code 143. 
[2022-12-04 02:25:21.195]Killed by external signal
.
22/12/04 02:25:21 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 32 on hub-

,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,...,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_copyright,withheld_in_countries
0,91070709,0,73919181,0,84084663,73532698,0,0,0,91070709,...,24778642,31410041,0,0,0,0,0,0,91182863,91156833


In [13]:
np.array(df_nullcount)

array([[91070709,        0, 73919181,        0, 84084663, 73532698,
               0,        0,        0, 91070709,        0,        0,
        76670768, 77069315, 77069315, 76670768, 76670768,        0,
               0, 90365800, 73767250,        0, 83161870, 83148064,
        83148064, 83161870, 83161870,        0,        0,        0,
        24778642, 31410041,        0,        0,        0,        0,
               0,        0, 91182863, 91156833]])

In [14]:
#Selecting only the columns needed for Analysis (Discarding the columns that have null values>65%)
cols_final = [
 'created_at',
 'display_text_range',
 'entities',
 'favorite_count',
 'favorited',
 'filter_level',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_from',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'tweet_text',
 'user']

#removed: 'coordinates', 'id','id_str','display_text_range','extended_entities','extended_tweet','geo','withheld_copyright','withheld_in_countries','in_reply_to_screen_name','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id','in_reply_to_user_id_str'
#'quoted_status','quoted_status_id','quoted_status_id_str','quoted_status_permalink','quoted_text'
tw_df = tw_df.select(cols_final)

----> I discarded the columns that had more than 65% null values in it. This was done by checking the count of null values in each column. 

In [15]:
#These are the different keywords/synonyms/words similar to education and education related tweets. 
keywords = ['children', 'kindergarten', 'testing', 'grade', 'student', 'grades', 
            'school', 'education', 'teachers', 'schools', 'college', 'students','math', 'science', 'primary_school', 'elementary_school', 'high_school', 'pre_school', 
                  'middle_school', 'k12', 'k-12', 'book', 'curriculum', 'learning', 'colleges', 'tuition']

In [16]:
tw_df_filt = tw_df.filter(
                          (lower(col('text')).contains("children"))\
                          | (lower(col('text')).contains("kindergraden"))\
                          | (lower(col('text')).contains("testing"))\
                          | (lower(col('text')).contains("grade"))\
                          | (lower(col('text')).contains("student"))\
                          | (lower(col('text')).contains("students"))\
                          | (lower(col('text')).contains("grades"))\
                          | (lower(col('text')).contains("school"))\
                          | (lower(col('text')).contains("education"))\
                          | (lower(col('text')).contains("k12"))\
                          | (lower(col('text')).contains("high school"))\
                          | (lower(col('text')).contains("college"))\
                          | (lower(col('text')).contains("elementary school"))\
                          | (lower(col('text')).contains("private education"))\
                          | (lower(col('text')).contains("public eduacation"))\
                          | (lower(col('text')).contains("online education"))\
                          | (lower(col('text')).contains("math"))\
                          | (lower(col('text')).contains("science"))\
                          | (lower(col('text')).contains("k-12"))\
                          | (lower(col('text')).contains("middle school"))
                          | (lower(col('text')).contains("curriculum"))\
                          | (lower(col('text')).contains("learning"))\
                          | (lower(col('text')).contains("colleges"))\
                          | (lower(col('text')).contains("tuition"))\
                          | (lower(col('text')).contains("primary school"))
                          
                          & ~(lower(col('text')).contains("shot"))\
                          & ~(lower(col('text')).contains("kill"))\
                          & ~(lower(col('text')).contains("murder"))\
                          & ~(lower(col('text')).contains("murdered"))\
                          & ~(lower(col('text')).contains("shoot"))\
                          & ~(lower(col('text')).contains("killed"))\
                          & ~(lower(col('text')).contains("killing"))\
                          & ~(lower(col('text')).contains("attack"))\
                          & ~(lower(col('text')).contains("attacked"))\
                          & ~(lower(col('text')).contains("gun"))\
                          & ~(lower(col('text')).contains("weapon"))\
                          & ~(lower(col('text')).contains("sex"))\
                          & ~(lower(col('text')).contains("porn"))\
                          & ~(lower(col('text')).contains("tounament"))\
                          & ~(lower(col('text')).contains("weapons"))\
                          & ~(lower(col('text')).contains("massacre"))\
                          & ~(lower(col('text')).contains("dead"))
                          )
                          


In [ ]:
print(tw_df_filt.count(),len(tw_df_filt.columns))

22/12/04 02:45:06 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_42_piece0 !
22/12/04 02:45:06 WARN org.apache.spark.storage.BlockManagerMaster: Failed to remove broadcast 42 with removeFromMaster = true - org.apache.spark.SparkException: Could not find BlockManagerEndpoint1.
	at org.apache.spark.rpc.netty.Dispatcher.postMessage(Dispatcher.scala:176)
	at org.apache.spark.rpc.netty.Dispatcher.postRemoteMessage(Dispatcher.scala:136)
	at org.apache.spark.rpc.netty.NettyRpcHandler.receive(NettyRpcEnv.scala:684)
	at org.apache.spark.network.server.TransportRequestHandler.processRpcRequest(TransportRequestHandler.java:161)
	at org.apache.spark.network.server.TransportRequestHandler.handle(TransportRequestHandler.java:109)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:140)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:53)
	at io.ne

63398669 22


--->I have discarded the irrelavent tweets and also the education related tweets that contain bad words/violence in it as it is not relavent for our analysis. 

In [18]:
tw_df.write.format("parquet").mode('overwrite').save('gs://msca-bdp-students-bucket/shared_data/suyashlakhani/tweets')

22/12/04 03:18:00 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670090078315_0009_01_000062 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wgps.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 03:18:00.875]Container killed on request. Exit code is 143
[2022-12-04 03:18:00.876]Container exited with a non-zero exit code 143. 
[2022-12-04 03:18:00.876]Killed by external signal
.
22/12/04 03:18:00 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 60 for reason Container from a bad node: container_1670090078315_0009_01_000062 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wgps.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 03:18:00.875]Container killed on request. Exit code is 143
[2022-12-04 03:18:00.876]Container exited with a non-zero exit code 143. 
[2022-12-04 03:18:00.876]Killed by external signal
.
22/12/04 03:

---> I have saved the filtered sparkdf as a parquet file for easier access in the next preprocessing step. 